## 10장 시퀀스 해킹, 해시, 슬라이스

이 장에서는 앞의 vector2d 모델을 더 업그레이드 하여 시퀀스 프로토콜을 구현할 것이다.

기본 시퀀스 프로토콜 (getitem 및 len 등) 및 안전한 표현, 슬라이싱, 해싱 등에 대한 기능을 구현한다.

In [9]:
from array import array
import reprlib
import math


class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)  # <1>

    def __iter__(self):
        return iter(self._components)  # <2>

    def __repr__(self):
        components = reprlib.repr(self._components)  # <3>
        components = components[components.find('['):-1]  # <4>
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))  # <5>

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))  # <6>

    def __bool__(self):
        return bool(abs(self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)  # <7>

### 프로토콜과 덕 타이핑

- python에서는 원하는 동작(시퀀스)을 위해 특별한 클래스를 상속할 필요가 없다. 단지 그러한 프로토콜을 따르는 특별 메서드를 구현하면 된다.
- python 객체는 슈퍼클래스가 무엇인지는 중요하지 않다. 단지 필요한 메서드만 구현하면 어떠한 클래스도 시퀀스가 필요한 곳에 사용될 수 있다.

In [1]:
import collections

Card = collections.namedtuple('Card', 'rank suit')

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self) -> None:
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, index):
        return self._cards[index]

위와같은 코드는 어디에도 시퀀스 프로토콜을 따른다고 적혀있지 않다. 하지만 파이썬 프로그래머들은 이 객체가 시퀀스인 것을 안다. 왜냐하면 이 객체가 시퀀스처럼 동작하기 때문에 시퀀스인 것이다. 이러한 매커니즘이 바로 덕 파이핑이다

In [7]:
deck = FrenchDeck()
next(iter(deck))

Card(rank='2', suit='spades')

In [8]:
deck[2:4]

[Card(rank='4', suit='spades'), Card(rank='5', suit='spades')]

In [18]:
from array import array
import numbers
import reprlib
import math


class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)  # <1>

    def __iter__(self):
        return iter(self._components)  # <2>

    def __repr__(self):
        components = reprlib.repr(self._components)  # <3>
        components = components[components.find('['):-1]  # <4>
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))  # <5>

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))  # <6>

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = f'{cls.__name__} indices must be integers'
            raise TypeError(msg)

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)  # <7>

    shortcut_names = 'xyzt'

    def __getattr__(self, name):
        cls = type(self)  # <1>
        if len(name) == 1:  # <2>
            pos = cls.shortcut_names.find(name)  # <3>
            if 0 <= pos < len(self._components):  # <4>
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'  # <5>
        raise AttributeError(msg.format(cls, name))

In [16]:
vector = Vector([1,2,3])
vector

Vector([1.0, 2.0, 3.0])

In [17]:
vector[0:2]

Vector([1.0, 2.0])

In [19]:
v = Vector(range(5))
v

Vector([0.0, 1.0, 2.0, 3.0, 4.0])

In [20]:
v.x

0.0

In [21]:
v.x = 10
v.x

10

In [22]:
v

Vector([0.0, 1.0, 2.0, 3.0, 4.0])